In [2]:
import pandas as pd 

In [2]:
df = pd.read_csv('../data/age.csv', encoding= 'cp949')
df.drop(columns=['Unnamed: 0'], axis= 0, inplace= True)

In [3]:
df= df.dropna(subset=['연령/시도'])
df = df[df['연령/시도'] != '계']  # '계' 값 삭제

In [4]:
df.drop(columns=['총계'], axis=0, inplace= True)

In [5]:
grouped_df = df.groupby(['성별', '연령/시도']).sum().reset_index()

In [6]:
grouped_df.drop(columns=['Year'], axis=0, inplace= True)

In [7]:
grouped_df

,성별,연령/시도,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주
0,기타,법인 및 사업자,5404194,4352157,1938280,6264180,944749,1006929,606326,130838,8293524,818176,1158948,1338907,1161428,3695567,1524931,3924825,4277576
1,남성,10대이하,25620,6760,2564,5787,1946,3174,2493,922,31459,3041,4496,4515,4375,3950,3177,7892,2483
2,남성,20대,615634,312817,285101,382307,225532,219591,169955,41906,1769254,251111,293567,378002,253760,283978,381736,427971,83596
3,남성,30대,3718865,1484789,1310712,1753618,799881,852202,670522,293311,8286638,821430,1013071,1421144,842443,900547,1387935,1777087,349723
4,남성,40대,5944825,2535996,2272107,2719880,1398336,1331640,1195272,533735,13535234,1399033,1605112,2428394,1667740,1737193,2598941,3395682,709065
5,남성,50대,6416554,2731260,2780405,2892606,1599302,1593618,1514631,428317,14366513,1970420,2118391,2832232,2341559,2557810,3686169,4253955,860458
6,남성,60대,5141228,2414633,2142044,2227311,1056908,1167267,1215537,255067,10143762,1809165,1783794,2299860,1931991,2181265,3372876,3478676,622502
7,남성,70대,1887543,799826,684754,565967,317183,340864,299252,67927,2777557,599956,548066,764197,697356,769381,1184468,1050642,207311
8,남성,80대,335436,111923,105074,73684,46851,50251,32388,9192,463673,113841,90183,124296,113135,114211,181238,140125,40699
9,남성,90대이상,30858,10957,8143,5825,3322,3329,2143,592,33860,5651,4406,8030,9271,8512,10857,9611,2664


In [ ]:
# python에 데이터 넣기
from mysql import connector
args = {
    'host' : 'localhost',
    'user' : 'root',
    'password' : 'root_1234',
    'port' : 3306
    }
conn = connector.connect(**args)  
cursor = conn.cursor(buffered=True)
# 도시별 데이터 삽입 
for idx, row in grouped_df.iterrows():
    for city in grouped_df.columns[2:]:
        sql = """
            select id from national_vehicle.region_name
            where city_do = %s
        """
        cursor.execute(sql, (city,))
        region_id = cursor.fetchone()[0]
        insert_query = """
                insert into national_vehicle.gender_age(gender, age_group, 
                                    region_id, population) values(%s, %s, %s, %s)
        """        
        cursor.execute(insert_query, (row['성별'], row['연령/시도'], region_id, row[city]))
conn.commit()
conn.close   
cursor.close() 

True

In [1]:
a = [('기타', '법인 및 사업자', 5404194), ('남성', '10대이하', 25620), ('남성', '20대', 615634), ('남성', '30대', 3718865), ('남성', '40대', 5944825), ('남성', '50대', 6416554), ('남성', '60대', 5141228), ('남성', '70대', 1887543), ('남성', '80대', 335436), ('남성', '90대이상', 30858), ('여성', '10대이하', 14051), ('여성', '20대', 313872), ('여성', '30대', 1507843), ('여성', '40대', 2243599), ('여성', '50대', 2427283), ('여성', '60대', 1629202), ('여성', '70대', 454506), ('여성', '80대', 83839), ('여성', '90대이상', 15189)]

In [ ]:
pd.DataFrame(a)

,0,1,2
0,기타,법인 및 사업자,5404194
1,남성,10대이하,25620
2,남성,20대,615634
3,남성,30대,3718865
4,남성,40대,5944825
5,남성,50대,6416554
6,남성,60대,5141228
7,남성,70대,1887543
8,남성,80대,335436
9,남성,90대이상,30858
